In [1]:
import requests
from bs4 import BeautifulSoup
HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36', 
           'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9'}

In [63]:
from lxml import html
import requests
import re
import pandas as pd

In [54]:
url = 'https://agroru.com/doska/udobreniya/'
HOST = 'https://agroru.com'

In [110]:
prods_url_list = []

for i in range(1, 72):
    url = f'https://agroru.com/doska/udobreniya/ct-0-p{i}.htm'

    q = requests.get(url)
    result = q.content
    tree = html.fromstring(q.content)

    soup = BeautifulSoup(result, 'lxml')
    prods = soup.find_all(class_='dl_item')

    for prod in prods:
        prod_page_url = HOST + prod.find('div', class_='dl_t_desc').find('div', class_='dl_t_hdr').find('a').get('href')
        prod_title = prod.find('div', class_='dl_t_desc').find('div', class_='dl_t_hdr').find('a').get('title')
        prod_location = prod.find('div', class_='dl_t_desc').find('div', class_='dl_t_text').find('div', class_='dl_shop_r').get_text(strip=True).replace('Местоположение товара:', '')
        prod_price = str(prod.find('div', class_='dl_shop').find('div', class_='dl_t_price_new')).replace('<div class="dl_t_price_new">\n\t\t\t\t\t\t', '').replace('р.\t\t\t\t\t\t\t\t\t\t\t\t\t<br/><span class="dl_t_edizm">', '').replace('</span>\n</div>', '')
        price = prod_price.split(' за')[0]
        izmer = prod_price.replace(price+' ', '')
        prod_seller = prod.find('div', class_='dl_shop').find('div', class_='dl_contacts').get_text(strip=True).replace('Продавец', '').replace('- частник', '')
        try:
            seller_info = HOST + prod.find('div', class_='dl_shop').find('div', class_='dl_contacts').find('a').get('href')
        except AttributeError:
            seller_info = HOST + prod.find('div', class_='dl_shop').find('div', class_='dl_shop_m').find('a').get('href')
        
        try:
            date = prod.find('div', class_='dl_t_desc').find('div', class_='dl_t_text').find('div', class_='dl_date').get_text(strip=True).replace('-', '.')
        except AttributeError:
            date = tree.xpath('/html/body/div[1]/div[2]/div/div[5]/div[2]/div[29]/div[3]/div[2]/div[4]/text()')
        prods_url_list.append({'title': prod_title, 'price': price.replace(' ', ''), 'izmer': izmer, 'location': prod_location, 'seller': prod_seller, 'seller_link': seller_info, 'date': date, 'link': prod_page_url})




In [111]:
with open('prods_list.txt', 'a') as file:
    for line in prods_url_list:
        file.write(f'{line}\n')

In [112]:
dataset = pd.DataFrame(prods_url_list)

In [113]:
dataset

,title,price,izmer,location,seller,seller_link,date,link
0,Известь негашеная ГОСТ,4800.00,за Тонну,"Р-ка Башкортостан, Уфа",Урал Робертович Гиниятуллин,https://agroru.com/pages/84613/,12.12.2020,https://agroru.com/doska/izvest-negashenaya-go...
1,Подкормка озимых пневмоходами,250.00,за Гектара,"Ростовская область, Миллерово","Темпагро, ООО",https://agroru.com/pages/316562/,12.12.2020,https://agroru.com/doska/podkormka-ozimyh-pnev...
2,Удобрение акварин газонный 1 кг.,300.00,за Штуку,"Москва, Москва","МосАгроГрупп, ООО",https://agroru.com/pages/135197/,12.12.2020,https://agroru.com/doska/udobrenie-akvarin-gaz...
3,Удобрение весна - лето 5 кг,370.00,за Штуку,"Москва, Москва","МосАгроГрупп, ООО",https://agroru.com/pages/135197/,12.12.2020,https://agroru.com/doska/udobrenie-amp-quot-ve...
4,"Удобрение экспресс - ремонт 2,5 кг",240.00,за Штуку,"Москва, Москва","МосАгроГрупп, ООО",https://agroru.com/pages/135197/,12.12.2020,https://agroru.com/doska/udobrenie-amp-amp-quo...
...,...,...,...,...,...,...,...,...
2823,Супер фосфат (фосфор гранулированный),17000.00,за Тонну,"Самарская область, Тольятти",ОЛЬГА,https://agroru.com/pages/175640/,,https://agroru.com/doska/fosfor-granulirovanny...
2824,Диаммофоска 10:26:26,26200.00,за Тонну,"Самарская область, Тольятти",ОЛЬГА,https://agroru.com/pages/175640/,,https://agroru.com/doska/diammofoska-32892.htm
2825,нитроаммофоска 16 16 16,22600.00,за Тонну,"Самарская область, Тольятти",ОЛЬГА,https://agroru.com/pages/175640/,,https://agroru.com/doska/nitroammofoska-16-16-...
2826,нитроаммофоска 22:7:12,22420.00,за Тонну,"Самарская область, Тольятти",ОЛЬГА,https://agroru.com/pages/175640/,,https://agroru.com/doska/nitroammofoska-22-7-1...


In [127]:
dataset.to_csv('dataset.csv', sep=',', encoding='utf-8')

In [134]:
from sqlalchemy import create_engine

In [135]:
engine = create_engine('sqlite:///save_pandas.db', echo=True)
sqlite_connection = engine.connect()

2020-12-13 02:15:46,220 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-13 02:15:46,222 INFO sqlalchemy.engine.base.Engine ()
2020-12-13 02:15:46,225 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-13 02:15:46,226 INFO sqlalchemy.engine.base.Engine ()


In [137]:
sqlite_table = "product"
dataset.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2020-12-13 02:17:19,441 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("product")
2020-12-13 02:17:19,442 INFO sqlalchemy.engine.base.Engine ()
2020-12-13 02:17:19,443 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("product")
2020-12-13 02:17:19,444 INFO sqlalchemy.engine.base.Engine ()
2020-12-13 02:17:19,447 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE product (
	"index" BIGINT, 
	title TEXT, 
	price TEXT, 
	izmer TEXT, 
	location TEXT, 
	seller TEXT, 
	seller_link TEXT, 
	date TEXT, 
	link TEXT
)


2020-12-13 02:17:19,447 INFO sqlalchemy.engine.base.Engine ()
2020-12-13 02:17:19,452 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-13 02:17:19,452 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_product_index ON product ("index")
2020-12-13 02:17:19,453 INFO sqlalchemy.engine.base.Engine ()
2020-12-13 02:17:19,457 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-13 02:17:19,470 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-13 02:17:19,738 I